In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
import pandas as pd
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.utils import np_utils
import seaborn as sns
from keras.applications.inception_v3 import preprocess_input


In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
MAIN_DIR = '/content/celeba-dataset/'
IMAGES = MAIN_DIR + 'img_align_celeba/img_align_celeba/'

#variables
IMG_SIZE = 100
IMG_SHAPE = (64,128,3)


In [ ]:
# import the data set that include the attribute for each picture
attributes = pd.read_csv(MAIN_DIR + 'list_attr_celeba.csv')
# set the index to the first column
attributes.set_index('image_id', inplace=True)
#replace -1 with 0
attributes.replace(to_replace=[-1], value=0, inplace=True)
attributes.shape

In [ ]:
# available attributes
attributes.columns


In [ ]:
# sample picture with the attributes
image_sample = load_img(IMAGES + '000001.jpg', target_size=(IMG_SIZE, IMG_SIZE))
plt.imshow(image_sample)


In [ ]:
# count of the number of female and male
plt.title('Count of Male and female')
sns.countplot(y='Male', data=attributes, color="c")
plt.show()

In [ ]:
# load recommended partitions
partitions = pd.read_csv(MAIN_DIR+'list_eval_partition.csv')
partitions.head()


In [ ]:
# display count by partition where
# 0 -> TRAINING
# 1 -> VALIDATION
# 2 -> TEST

partitions['partition'].value_counts().sort_index()

In [ ]:
# join the partirions with the attributes
partitions.set_index('image_id', inplace=True)
partitions_attributes = partitions.join(attributes['Male'], how='inner')
partitions_attributes.head()

In [ ]:
# load reshape image

def load_reshape_img(fname):
    img = load_img(fname)
    x = img_to_array(img)/255.
    x = x.reshape((1,) + x.shape)

    return x


def generate_df(partition, attr, num_samples):
    
    df_ = partitions_attributes[(partitions_attributes['partition'] == partition) 
                           & (partitions_attributes[attr] == 0)].sample(int(num_samples/2))
    df_ = pd.concat([df_,
                      partitions_attributes[(partitions_attributes['partition'] == partition) 
                                  & (partitions_attributes[attr] == 1)].sample(int(num_samples/2))])

    # for Train and Validation
    if partition != 2:
        x_ = np.array([load_reshape_img(IMAGES + fname) for fname in df_.index])
        x_ = x_.reshape(x_.shape[0], 218, 178, 3)
        y_ = np_utils.to_categorical(df_[attr],2)
    # for Test
    else:
        x_ = []
        y_ = []

        for index, target in df_.iterrows():
            im = cv2.imread(IMAGES + index)
            im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE)).astype(np.float32) / 255.0
            im = np.expand_dims(im, axis =0)
            x_.append(im)
            y_.append(target[attr])

    return x_, y_

In [ ]:
# Generate image generator for data augmentation
datagen =  ImageDataGenerator(
  #preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True
)



In [ ]:
# apply histogram of oriented gradients to the image
def hog_features(img, orient, pix_per_cell, cell_per_block, block_norm="L2-Hys"):
    # Call with two outputs if you are training the model:
    hog_features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                 cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                 visualise=True, feature_vector=False, block_norm=block_norm)
    return hog_features

In [ ]:
# apply histogram of oriented gradients to the image
def hog_features_img(img, orient, pix_per_cell, cell_per_block, block_norm="L2-Hys"):
    # Call with two outputs if you are training the model:
    hog_features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                 cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                 visualise=True, feature_vector=False, block_norm=block_norm)
    # 
    return hog_features, hog_image

In [ ]:
# load one image and reshape
img = load_img(IMAGES + '000001.jpg')
x = img_to_array(img)/255.
x = x.reshape((1,) + x.shape)

# plot 10 augmented images of the loaded iamge
plt.figure(figsize=(20,10))
plt.suptitle('Data Augmentation', fontsize=28)

i = 0
for batch in datagen.flow(x, batch_size=1):
    plt.subplot(3, 5, i+1)
    plt.grid(False)
    plt.imshow( batch.reshape(218, 178, 3))
    
    if i == 9:
        break
    i += 1
    
plt.show()

In [ ]:
# Train data
x_train, y_train = generate_df(0, 'Male', 10000)

In [ ]:

# Training,  Data Preparationm  Data Augmentation with generators
train_datagen =  ImageDataGenerator(
  preprocessing_function=preprocess_input,
  rotation_range=30,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
)

train_datagen.fit(x_train)

train_generator = train_datagen.flow(
x_train, y_train,
batch_size=16,
)



In [ ]:
# validation data
x_valid, y_valid = generate_df(1, 'Male', 2000)

In [ ]:
# 

In [ ]:
# building the model
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
# SVM
svc = SVC(kernel='linear', C=1, probability=True)

# pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
    ('svc', svc)
])

In [ ]:
# fit the model
pipeline.fit(x_train, y_train)

# y should be a 1d array, got an array of shape (1000, 2) instead.
# fix this by flattening the array
y_train = y_train.flatten()
y_valid = y_valid.flatten()


In [ ]:
# building the model
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# test split
from sklearn.model_selection import train_test_split

In [ ]:
# test split
features_train, features_test, labels_train, labels_test = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
linear_svm = LinearSVC(random_state=42)
linear_svm_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
    ('svc', linear_svm)
])

#  Found array with dim 4. StandardScaler expected <= 2.
# fix this by flattening the array
features_train = features_train.reshape(features_train.shape[0], -1)
features_test = features_test.reshape(features_test.shape[0], -1)


In [ ]:
linear_svm_pipeline.fit(features_train, labels_train)

In [ ]:
# build a CNN model to classify the images
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# build the model
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))